MURA Pre-train

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
input_size = (320, 320)
n_classes = 2

In [ ]:
#@title Paths definition
data_folder = 'data/'
model_folder= 'models/'

In [ ]:
#@title Function to load data from hdf5
import h5py
import numpy as np

def import_from_hdf5(path, verbose=False):
    with h5py.File(path, "r", driver='core') as hdf5_file:

        train_data = None
        batch_begin, batch_end = 0, 200
        train_samples = len(hdf5_file['train_img'])
        while True:
            batch_end = min(batch_end, train_samples)
            batch = hdf5_file['train_img'][batch_begin:batch_end, :, :, :]
            if train_data is None:
                train_data = batch
            else:
                train_data = np.concatenate([train_data, batch], axis=0)
            if verbose:
                print('Loaded', batch_end, 'samples of', train_samples, 'samples of train data')
            if batch_end == train_samples:
                break
            else:
                batch_begin = batch_end
                batch_end += 200

        train_labels = hdf5_file['train_labels'][:]

        valid_data = None
        batch_begin, batch_end = 0, 200
        val_samples = len(hdf5_file['val_img'])
        while True:
            batch_end = min(batch_end, val_samples)
            batch = hdf5_file['val_img'][batch_begin:batch_end, :, :, :]
            if valid_data is None:
                valid_data = batch
            else:
                valid_data = np.concatenate([valid_data, batch], axis=0)
            if verbose:
                print('Loaded', batch_end, 'samples of', val_samples, 'samples of val data')
            if batch_end == val_samples:
                break
            else:
                batch_begin = batch_end
                batch_end += 200

        valid_labels = hdf5_file['val_labels'][:]

        train_data, valid_data = train_data[:, :, :, :], valid_data[:, :, :, :]

    return train_data, train_labels, valid_data, valid_labels

In [ ]:
#@title Load hand data
hand_train, hand_train_labels, hand_valid, hand_valid_labels = import_from_hdf5(data_folder + 'hand.hdf5')

print(hand_train.shape, hand_train_labels.shape, hand_valid.shape, hand_valid_labels.shape)

In [ ]:
#@title Visualize some samples
import matplotlib.pyplot as plt

shift = 1520
n_cols, n_rows = 4, 3

fig, ax = plt.subplots(figsize=(5 * n_cols, 5 * n_rows), ncols=n_cols, nrows=n_rows)

for i in range(n_rows):
    for k in range(n_cols):
        label = 'Normal' if hand_train_labels[shift + i * n_cols + k] == 0 else 'Abnormal'
        ax[i, k].imshow(hand_train[shift + i * n_cols + k].astype(np.uint8))
        ax[i, k].set_xlabel(label)

plt.show()

In [ ]:
#@title Model definition
#Model definition
import os
import tempfile
import keras
from keras import backend as K
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201, preprocess_input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers
from tensorflow.keras.models import model_from_json

K.set_learning_phase(1) #?

#Hack to add a regularization to a pre-trained model
#https://sthalles.github.io/keras-regularizer/
def add_regularization(model, regularizer=tf.keras.regularizers.l2(0.0001)):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)
    #tmp_weights_path = model_folder + 'tmp_weights.h5'
    #model.save_weights(tmp_weights_path, overwrite=True)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

def get_compiled_model_instance(lr=0.0001, version=121, pooling='avg', optimizer='sgd', beta1=0.9, beta2=0.999,
                                regularizer=None, reg_lambda=0.01, label_smoothing=0.0):

    def loss(y_true, y_pred): #Wrapping a tf loss to the keras interface with only 2 args
        return binary_crossentropy(y_true, y_pred, label_smoothing=label_smoothing)
    
    pure_loss=loss #Will be passed as a metric to be watched as a "pure" (without regularization) loss

    def get_lr_metric(optimizer): #Custom metric to monitor learning rate
        def lr(y_true, y_pred):
            return optimizer.lr
        return lr
    
    if regularizer=='l1':
        regularizer=regularizers.l1(reg_lambda)
    elif regularizer=='l2':
        regularizer=regularizers.l2(reg_lambda)
    elif not regularizer:
        pass
    elif len(reg_lambda)==2:
        l1, l2 = reg_lambda
        regularizer=regularizers.l1_l2(l1=l1, l2=l2)
    else:
        raise VaelueError('Unknown regularizer')
    
    base_model = None
    if version==121:
        base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=None, pooling=pooling)
    elif version==169:
        base_model = DenseNet169(include_top=False, weights='imagenet', input_shape=None, pooling=pooling)
    elif version==201:
        base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=None, pooling=pooling)
    else:
        raise ValueError('Unknown DenseNet version')

    for layer in base_model.layers: layer.trainable = True #Is this necessary?
    
    if regularizer:
        base_model=add_regularization(base_model, regularizer=regularizer)

    x = base_model.output
    predictions = Dense(1, activation='sigmoid')(x)
    #predictions = Dense(n_classes, activation='softmax')(x)

    if optimizer=='adam':
        optimizer=Adam(lr=lr, beta_1=beta1, beta_2=beta2)
    elif optimizer=='sgd':
        optimizer=SGD(lr=lr, momentum=beta1, nesterov=True)
    else:
        raise ValueError('Unknown optimizer')

    lr_metric = get_lr_metric(optimizer)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy', lr_metric, pure_loss])
    
    

    return model

#get_compiled_model_instance()

In [ ]:
#@title Data generators definition
#Data generators

#Warning, RAM-eager!

from keras.preprocessing.image import ImageDataGenerator

#train_data, val_data = preprocess_input(hand_train), preprocess_input(hand_valid)

#train_datagen = ImageDataGenerator(
#    horizontal_flip=True, #Augment
#    vertical_flip=True,
#    rotation_range=90,
#    #width_shift_range=0.2,
#    #height_shift_range=0.2
#    )

#valid_datagen = ImageDataGenerator()

train_datagen = ImageDataGenerator(
    featurewise_center = True, #Mean-shift
    featurewise_std_normalization = True, #Standardization
    horizontal_flip=True, #Augment
    vertical_flip=True,
    rotation_range=90,
    #width_shift_range=0.2,
    #height_shift_range=0.2
    )

train_datagen.fit(hand_train)

valid_datagen = ImageDataGenerator( #Data standardization, no augment
    featurewise_center=True,
    featurewise_std_normalization = True,
)

valid_datagen.fit(hand_valid)

In [ ]:
#Learning rate scheduler - reduce by factor 0.1 on plateau longer than 10 epochs

from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_delta=0.001, verbose=1, min_lr=1e-7)

In [ ]:
#@title Class weights computing
#Class weights computing to penalize errors on more rare class

from sklearn.utils import class_weight

class_weight = class_weight.compute_class_weight('balanced', np.unique(hand_train_labels), hand_train_labels)

class_weight

In [ ]:
#@title Modular live loss plotter

#Modular live loss plotter for Keras models
#Allows to create custom layouts of per-batch or per-epoch plots for different metrics

#Monitor class defines a plot, which either may be batch or epoch-scoped, and may contain several graphs
#Batch monitor plots its values per batch, and refreshes itself on new epoch begin
#Epoch monitor plots its values per epoch, and performs no refresh
#All values/last N values displaying
#Log-scale/Linear scale displaying

#Plotter callback handles different Monitors and responds to the actual plotting
#Defines a grid where Monitors will be drawn, grid size, refresh rate in batches
#when the Monitors will be re-drawn in addition to per-epoch update
#Plotter can be silenced to disable plotting and only archivate per-epoch data

from IPython.display import clear_output
from keras.callbacks import Callback
import matplotlib.pyplot as plt

class Monitor():
    def __init__(self, scope='epoch', monitors= [ 'loss' ], plot_last=-1, log_scale=False, precision=4):
        self.scope = scope.lower()
        self.monitors = [ monitor.lower() for monitor in monitors ]
        self.plot_last = max(0, plot_last)
        self.x = []
        self.ys = [ [] for monitor in monitors ]
        self.log_scale = log_scale
        self.precision = precision

    def reinit(self):
        self.x = []
        self.ys = [ [] for monitor in self.monitors ]

    def update(self, iteration, logs={}):
        self.x.append(iteration)
        [ self.ys[i].append(logs.get(monitor)) for i, monitor in enumerate(self.monitors) ]

    def plot(self, axis):
        x_data = self.x[ -self.plot_last : ]
        y_array = [ y_data[ -self.plot_last : ] for y_data in self.ys ]

        for i, y_data in enumerate(y_array):
            label = self.monitors[i] + '_' + self.scope
            if self.log_scale:
                axis.set_yscale('log')
            axis.plot(x_data, y_data, label=label)
            if self.precision > 0 and len(x_data) > 0:
                text = str(round(y_data[-1],  self.precision))
                axis.text(x_data[-1], y_data[-1], text)
            axis.legend()


class Plotter(Callback):
    def __init__(self, scale=5, n_cols=2, n_rows=1, monitors=[], refresh_rate=-1, silent=False):
        if (n_cols * n_rows < len(monitors)):
            raise ValueError('Grid is too small to fit all monitors!')

        self.n_cols = n_cols
        self.n_rows = n_rows
        self.scale = scale

        self.monitors = monitors

        self.batch_monitors, self.epoch_monitors = [], []

        for monitor in monitors:
            if monitor.scope == 'epoch':
                self.epoch_monitors.append(monitor)
            elif monitor.scope == 'batch':
                self.batch_monitors.append(monitor)

        self.refresh_rate = refresh_rate
        self.silent = False

    def on_train_begin(self, logs={}):
        pass

    def on_epoch_begin(self, epoch, logs={}):
        [ monitor.reinit() for monitor in self.batch_monitors ]

    def plot(self):
        clear_output(wait=True)

        figsize = ( self.scale * self.n_cols, self.scale * self.n_rows)
        fig, ax = plt.subplots(figsize=figsize, ncols=self.n_cols, nrows=self.n_rows)

        for i in range(self.n_rows):
            for k in range(self.n_cols):
                index = i * self.n_cols + k
                axis = ax[i, k]
                self.monitors[index].plot(axis)

        plt.show()

    def on_batch_end(self, batch, logs={}):
        [ monitor.update(batch, logs) for monitor in self.batch_monitors ]

        if self.silent or batch == 0 or self.refresh_rate <= 0 or batch % self.refresh_rate != 0:
            return

        self.plot()

    def on_epoch_end(self, epoch, logs={}):
        [ monitor.update(epoch, logs) for monitor in self.epoch_monitors ]

        if self.silent:
            return

        self.plot()


In [ ]:
monitors = [
    Monitor(scope='batch', monitors = ['loss']),
    Monitor(scope='epoch', monitors = ['loss', 'val_loss']),
    Monitor(scope='epoch', monitors = ['loss', 'val_loss'], plot_last=16),
    
    #No-reg loss monitor
    Monitor(scope='epoch', monitors = ['pure_loss', 'val_pure_loss']),
    
    Monitor(scope='batch', monitors = ['accuracy'], precision=3),
    Monitor(scope='epoch', monitors = ['accuracy', 'val_accuracy'], precision=3),
    Monitor(scope='epoch', monitors = ['lr'], log_scale=False, precision=7), #lr represents custom metric defined to watch the learning rate
]

plotter = Plotter(monitors=monitors, n_rows=2, n_cols=4, refresh_rate=32)

In [ ]:
#@title Epoch counter used to monitor epochs through multiple training launches

class EpochCounter(Callback):
    def __init__(self, initial_epoch=0):
        self.epoch = initial_epoch

    def on_epoch_end(self, epoch, logs={}):
        self.epoch += 1

epoch_counter = EpochCounter()

In [ ]:
#TensorBoard as an advanced tool to watch the learning process

#%load_ext tensorboard.notebook

#from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

#tbc=TensorBoardColab()

In [ ]:
from keras.callbacks import ModelCheckpoint #Mandatory for a long-time training

#Validation loss-based checkpointing
#filepath=export_folder + "dn-mura-pretrain-{epoch:03d}-{val_loss:.4f}-{lr:.7f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', period=5)

#Validation accuracy-based checkpointing
filepath=model_folder + "dn-mura-pretrain-{epoch:03d}-{val_loss:.4f}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto', period=5)

In [ ]:
#@title Restarting lr callback
#Restarting LR with different slopes

class RestartingLR(Callback):
    def __init__(self, min_lr=0.0001, max_lr=0.001, warmup_t=0, decrease_t=50, warmup='linear', decrease='cosine',
               warmup_enlengthen=1.0, decrease_enlengthen=1.0, cycles_demphing=1.0, epoch_shift=0,
               checkpoint_on_cycle_end=False, path=None, weights_only=False, verbose=True):
    
        super(RestartingLR, self).__init__()
        self.min_lr=min_lr
        self.max_lr=max_lr
        self.decrease_t=decrease_t
        self.warmup_t=warmup_t

        #LR value functions depending on the passed epoch number from last restart squashed to the [0, 1] interval
        #Functions correspond to decrease, to compute increase with them use 1 - x instead
        functions =  {
            'linear' : lambda x, min_lr, max_lr : (min_lr - max_lr) * x + max_lr,
            'cosine' : lambda x, min_lr, max_lr : (max_lr - min_lr) * (np.cos(x * np.pi) + 1) * 1/2 + min_lr,

            #Linear descend of lr's exponents. If min_lr=1e-7 and max_lr=1e-3,
            #lr will be 10^f(x), where f(x) is a linear function with values descending from -3 to -7 
            'log' : lambda x, min_lr, max_lr: np.exp(np.log(min_lr/max_lr) * x + np.log(max_lr))
            }

        if warmup not in functions.keys(): raise ValueError('No such warmup function available')
        if decrease not in functions.keys(): raise ValueError('No such decrease function available')

        self.warmup=functions[warmup]
        self.decrease=functions[decrease]

        self.warmup_enlengthen=warmup_enlengthen
        self.decrease_enlengthen=decrease_enlengthen
        self.cycles_demphing=cycles_demphing

        if epoch_shift < 0: raise ValueError('Epoch shift should be >= 0')

        self.epoch_shift=epoch_shift

        self.checkpoint=checkpoint_on_cycle_end
        if path is None and checkpoint_on_cycle_end: raise ValueError('Empty path while checkpointing is on')

        self.path=path
        self.verbose=verbose
        self.weights_only=weights_only

    def lr_value(self, epoch=0):
        epoch+=self.epoch_shift
        cycles_passed, decrease_t, warmup_t = 0, self.decrease_t, self.warmup_t

        cycle_epoch=epoch

        while True:
            if cycle_epoch - (decrease_t + warmup_t) >= 0:
                cycle_epoch -= decrease_t + warmup_t
                decrease_t *= self.decrease_enlengthen
                warmup_t *= self.warmup_enlengthen
                decrease_t, warmup_t = int(decrease_t), int(warmup_t)
                cycles_passed += 1
            else:
                break

        lr, min_lr, max_lr = None, self.min_lr, self.min_lr + (self.max_lr - self.min_lr) * (self.cycles_demphing ** cycles_passed)

        if cycle_epoch < warmup_t:
            arg = 1 - cycle_epoch / warmup_t
            lr = self.warmup(arg, min_lr, max_lr)

        elif cycle_epoch < warmup_t + decrease_t:
            arg = (cycle_epoch - warmup_t) / decrease_t
            lr = self.decrease(arg, min_lr, max_lr)

        return lr
    
    def on_train_begin(self, logs={}):
        K.set_value(self.model.optimizer.lr, self.lr_value())

    def on_epoch_end(self, epoch, logs={}):
        if self.checkpoint and (self.model.optimizer.lr - self.min_lr) < 1e-10:
            if self.weights_only:
                self.mode.save_weights(self.path, overwrite=True)
            else:
                self.model.save(self.path, overwrite=True)
            if self.verbose:
                print('Saved model on the cycle end at epoch', epoch)
        lr = self.lr_value(epoch + 1)
        K.set_value(self.model.optimizer.lr, lr)

    def get_lr_values(self, epochs=100):
        return [ self.lr_value(i) for i in range(epochs) ]

       

In [ ]:
checkpoint_path = model_folder + "dn-mura-pretrain-cycle-{epoch:03d}-{val_loss:.4f}-{val_acc:.2f}.hdf5"

clr = RestartingLR(min_lr=1e-5, max_lr=1e-4, warmup_t=0, decrease_t=50, warmup='cosine', decrease='cosine',
                  warmup_enlengthen=1.0, decrease_enlengthen=1.0, cycles_demphing=1.0, epoch_shift=0,
                  checkpoint_on_cycle_end=True, path=checkpoint_path)
       

iterations = 200
plt.figure()
#plt.yscale('log')
plt.plot(range(iterations), clr.get_lr_values(iterations))
plt.show()

In [ ]:
callbacks = [
             checkpoint,
             #reduce_lr,
             clr,
             plotter,
             epoch_counter,
             #TensorBoardColabCallback(tbc),
             ]

In [ ]:
#@title Functions to load model from file
from tensorflow.keras.models import load_model

import re

def parse_filename(path):
    text = re.compile('[\w]+-(\d+)-([-+]?[0-9]*\.?[0-9]+)-([-+]?[0-9]*\.?[0-9]+).hdf5').findall(path)
    if len(text) == 0:
        return None
    text = text[0]
    return int(text[0]), float(text[1]), float(text[2])
  
def load_model_from_checkpoint(path, label_smoothing=0.0):
  
    def loss(y_true, y_pred): #Wrapping a tf loss to the keras interface with only 2 args
        return binary_crossentropy(y_true, y_pred, label_smoothing=label_smoothing)

    def get_lr_metric(optimizer): #Custom metric to monitor learning rate
        def lr(y_true, y_pred):
            return optimizer.lr
        return lr

    def placeholder_lr(x, y): #We need placeholder since lr metric definition requires model's optimizer
        return 1.0

    model = load_model(path, custom_objects={'loss' : loss, 'lr' : placeholder_lr})#Load the model with placeholder
    lr = get_lr_metric(model.optimizer) #Define the correct lr metric
    metrics = [ metric if metric.name != 'placeholder_lr' else lr for metric in model.metrics ] #Replace the placeholder with the true metric

    model.compile(loss=model.loss, #Recompile model
                optimizer=model.optimizer,
                metrics=metrics)

    return model

#parse_filename('drive/My Drive/Work/ML/RA2/dn121-mura-pretrain-020-0.79-0.0001000.hdf5')

#model = load_model_from_checkpoint('drive/My Drive/Work/ML/RA2/dn121-mura-pretrain-020-0.79-0.0001000.hdf5')

In [ ]:
#Get newly compiled pre-trained model with small initial lr
#to not disturb pre-trained weights too much
hand_model = get_compiled_model_instance(lr=0.0001, regularizer='l2', reg_lambda=1e-5)
hand_model.losses

#Load previously saved model checkpoint
#model_path = export_folder + 'dn121-mura-pretrain-020-0.79-0.0001000.hdf5'
#hand_model = load_model_from_checkpoint(model_path)
#epoch_counter.epoch = parse_filename(model_path)[0]

#print('Model checkpoint epoch', epoch_counter.epoch)

#Evaluate model to ensure that model was loaded and recompiled correctly
#print('Model evaluation:', hand_model.evaluate_generator(valid_datagen.flow(hand_valid, hand_valid_labels, batch_size=len(hand_valid))))

In [ ]:
#import tensorflow as tf

#!!! Workaround to avoid the FailedPreconditionError at the epoch end !!!

#K.set_session(tf.Session(graph=hand_model.output.graph))
#init = tf.global_variables_initializer()
#K.get_session().run(init)

In [ ]:
#hand_model.summary()

In [ ]:
histories = {}

In [ ]:
#Launch primary model training

batch_size = 8

val_batch_size=10
validation_steps= len(hand_valid_labels) // val_batch_size #Validate on full set

epochs = 50 + epoch_counter.epoch #For multiple launches

hand_model_history = hand_model.fit_generator(
    train_datagen.flow(hand_train, hand_train_labels, batch_size=batch_size, shuffle=True),
    validation_data=valid_datagen.flow(hand_valid, hand_valid_labels, batch_size=val_batch_size, shuffle=True), #Validation set is used only at the epoch end
    validation_steps=validation_steps,
    #train_datagen.flow(train_data, hand_train_labels, batch_size=batch_size, shuffle=True),
    #validation_data=valid_datagen.flow(val_data, hand_valid_labels, batch_size=val_batch_size, shuffle=True),
    class_weight=class_weight,
    epochs=epochs,
    callbacks=callbacks,
    initial_epoch=epoch_counter.epoch,
    verbose=2
)

histories[epoch_counter.epoch]=hand_model_history